## Top-level Global Var

In [1]:
DEBUG = 8

## Imports

In [2]:
import argparse
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np
import os
from os.path import dirname
import random
import torch
import torchvision
import tqdm

if DEBUG>0:
    from utils.models1 import Classifier
else:
    from utils.models import Classifier
from utils.dataset import NCaltech101
from utils.loader import Loader
from utils.train_eval import train_one_epoch, eval_one_epoch

if DEBUG>=8:
    if DEBUG==9:
        seed = 1586354275
    else:
        import time
        seed = int( time.time())
    print("Seed: %d" % seed)
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)

Seed: 1587549325


In [3]:
class FLAGS:
    height = 180
    width = 240
    batch_size = 8
    checkpoint = 'log/final/model_best.pth'
    device = 'cuda'
    test_dataset = 'N-Caltech101/testing'
    num_workers = 0
    pin_memory = True
    
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"----------------------------\n"
                f"Starting testing with \n"
                f"height: {self.height}\n"
                f"width: {self.width}\n"
                f"batch_size: {self.batch_size}\n"
                f"checkpoint: {self.checkpoint}\n"
                f"device: {self.device}\n"
                f"test_dataset: {self.test_dataset}\n"
                f"----------------------------")

In [4]:
flags = FLAGS()
dim = (flags.height, flags.width)

# datasets
test_dataset = NCaltech101(flags.test_dataset)
datasetClasses = test_dataset.getClasses()

# construct loader, handles data streaming to gpu
test_loader = Loader(test_dataset, flags, device=flags.device)

# model, and put to device
model = Classifier(device=flags.device, dimension=dim)
ckpt = torch.load(flags.checkpoint, map_location=flags.device)
model.load_state_dict(ckpt["state_dict"])
model = model.to(flags.device)

----------------------------
Starting testing with 
height: 180
width: 240
batch_size: 8
checkpoint: log/model_best.pth
device: cpu
test_dataset: N-Caltech101/testing
----------------------------


FileNotFoundError: [Errno 2] No such file or directory: 'log/model_best.pth'

In [ ]:
""" Visualization of test set """
itr = iter(test_loader)
events, labels = next(itr)
model.setMode(1)
preprocessed = model.quantization_layer.forward(events)
preprocessed = preprocessed.cpu().numpy()

print(labels)

fig = plt.figure(figsize=(18, 36))
ax = []
rows = 8
columns = 1
for i in range(rows * columns):
    ax.append( fig.add_subplot(rows, columns, i+1))
    ax[-1].set_title( datasetClasses[ labels[i].item()])
    plt.imshow(preprocessed[i][0], cmap='gray', vmin=0, vmax=1)
plt.tight_layout()
plt.show()

In [ ]:
model = model.eval()
model.setMode(1)
test_loss, test_accuracy = eval_one_epoch(model, flags.device, test_loader)
print(f"Test Loss {test_loss:.4f}  Accuracy {test_accuracy:.4f}")